<a href="https://colab.research.google.com/github/fahmisyuhada/GAN_with_Keras/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import tensorflow as tf
from keras.datasets import mnist


In [2]:
keras.__version__

'2.12.0'